In [1]:
import math
import random
import os
from pathlib import Path 
import time
from tqdm import tqdm

from sklearn.model_selection import train_test_split

import data_reader
import pickle

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

### define model

In [2]:
class NNClassifier(nn.Module):
    def __init__(self, hidden_num=1, dropout_p=None,
                 input_dim=174, hidden_dim=128, class_num=2):
        super(NNClassifier, self).__init__()
        # loss
        self.loss = nn.CrossEntropyLoss()
        # hidden-hidden fcs
        self.hiddens = [nn.Linear(input_dim, hidden_dim) for _ in range(hidden_num-1)]
        # insert input-hidden fc
        self.hiddens.insert(0, nn.Linear(input_dim, hidden_dim))
        # dropout layers
        self.dropout_p = dropout_p
        if dropout_p is not None:
            self.drops = [nn.Dropout(p=dropout_p) for _ in range(hidden_num)]
        # output layer
        self.out = nn.Linear(hidden_dim, class_num)
        # dropout
        
        
    def forward(self, x):
        for i in range(len(self.hiddens)):
            x = F.relu(self.hiddens[i](x))
            if self.dropout_p is not None:
                x = self.drops[i](x)
        x = self.out(x)
        x = x.squeeze()
        val, idx = torch.max(x, dim=1)
        return x, idx
    
    def compute_loss(self, pred_vec, gold_vec):
        return self.loss(pred_vec, gold_vec)

### wrap-up

In [3]:
def train_and_test_once(X_train, X_test, Y_train, Y_test,
               hidden_num, dropout_p, lr, epoch_num,
               label_index, debug_mode=True):
    debug_report_seg = epoch_num // 10
    train_size, input_dim = X_train.shape
    model = NNClassifier(input_dim=input_dim, dropout_p=dropout_p)
    optimizer = optim.SGD(model.parameters(), lr=0.5)
    #optim.Adam(model.parameters())
    optimizer.zero_grad()
    model.train()
    start_train = time.time()
    for epoch in range(epoch_num):
        optimizer.zero_grad()
        inputs = torch.tensor(X_train).float()
        golds  = torch.tensor(Y_train[:,label_index]).long()
        
        if debug_mode:
            print(golds)
            debug_mode = False
            
        pred_vals, pred_labels  = model(inputs)

        loss = model.compute_loss(pred_vals, golds)

        if debug_mode and epoch % debug_report_seg == 0:
            acc = golds.eq(pred_labels).sum().float() / train_size
            print("epoch {}, loss = {}, acc = {}".format(epoch, loss, acc))

        loss.backward()
        optimizer.step()

    acc = golds.eq(pred_labels).sum().float() / train_size
    if debug_mode:
        print("training: loss = {}, acc = {}".format(loss, acc))

    model.eval()
    test_size, input_dim = X_test.shape
    inputs = torch.tensor(X_test).float()
    golds  = torch.tensor(Y_test[:,label_index]).long()
    pred_vals, pred_labels  = model(inputs)
    loss = model.compute_loss(pred_vals, golds)
    acc = golds.eq(pred_labels).sum().float() / test_size
    if debug_mode:
        print("test: loss = {}, acc = {}".format(loss, acc))
    
    return model, loss, acc
    

In [4]:
def train_and_test_k_fold_cross_val(feature_path="train_features.pkl", label_path="train.csv", 
                                    filter_mode="", filter_val=0, k=10, seed=777, 
                                    hidden_num=1, dropout_p=None, lr=0.1, epoch_num=1000, debug_mode=True):
    best_ch_model = None
    best_ch_acc = 0
    total_ch_acc = 0
    best_en_model = None
    best_en_acc = 0
    total_en_acc = 0
    for i, data in enumerate(data_reader.k_fold_cross_val(
                                filter_mode=filter_mode, filter_val=filter_val,
                                feature_path=feature_path, 
                                label_path=label_path,
                                k=k, seed=seed)):
        train, test = data
        print(f"training set size={len(train)}, test set size={len(test)}")
        X_train = train[:, :-2]
        Y_train = train[:, -2:]
        X_test = test[:, :-2]
        Y_test = test[:, -2:]
        Y_train = (Y_train >= 0).astype(int)
        Y_test = (Y_test >= 0).astype(int)
        # ch
        model , loss, acc = train_and_test_once(X_train, X_test, Y_train, Y_test, 
                                                hidden_num, dropout_p, lr, epoch_num,
                                                label_index=0, debug_mode=debug_mode)
        total_ch_acc += acc
        if acc > best_ch_acc:
            best_ch_acc = acc
            best_ch_model = model
        ch_acc = acc
        
        # en
        model , loss, acc = train_and_test_once(X_train, X_test, Y_train, Y_test, 
                                                hidden_num, dropout_p, lr, epoch_num,
                                                label_index=1, debug_mode=debug_mode)
        total_en_acc += acc
        if acc > best_en_acc:
            best_en_acc = acc
            best_en_model = model
        en_acc = acc
        
        print(f"{i}-th fold: ch acc = {ch_acc}, en acc = {en_acc}")
    
    print(f"average ch acc = {total_ch_acc / k}, average en acc = {total_en_acc / k}")
    
    return best_ch_model, best_en_model

---

## data no filtered

### 1 layer, no dropout

In [5]:
model_ch, model_en = train_and_test_k_fold_cross_val(hidden_num=1, dropout_p=None, lr=0.5, epoch_num=1000, debug_mode=False)

0-th fold: ch acc = 0.6000000238418579, en acc = 0.6000000238418579
1-th fold: ch acc = 0.6499999761581421, en acc = 0.5333333611488342
2-th fold: ch acc = 0.5666666626930237, en acc = 0.5166666507720947
3-th fold: ch acc = 0.550000011920929, en acc = 0.5833333134651184
4-th fold: ch acc = 0.5833333134651184, en acc = 0.5666666626930237
5-th fold: ch acc = 0.5666666626930237, en acc = 0.6000000238418579
6-th fold: ch acc = 0.6666666865348816, en acc = 0.6499999761581421
7-th fold: ch acc = 0.699999988079071, en acc = 0.5333333611488342
8-th fold: ch acc = 0.5666666626930237, en acc = 0.5333333611488342
9-th fold: ch acc = 0.7333333492279053, en acc = 0.550000011920929
average ch acc = 0.6183332204818726, average en acc = 0.5666667222976685


### 3 layer, no dropout

In [6]:
model_ch, model_en = train_and_test_k_fold_cross_val(hidden_num=3, dropout_p=None, lr=0.5, epoch_num=1000, debug_mode=False)

0-th fold: ch acc = 0.6000000238418579, en acc = 0.6333333253860474
1-th fold: ch acc = 0.6666666865348816, en acc = 0.6000000238418579
2-th fold: ch acc = 0.5833333134651184, en acc = 0.6000000238418579
3-th fold: ch acc = 0.5833333134651184, en acc = 0.6333333253860474
4-th fold: ch acc = 0.5666666626930237, en acc = 0.6499999761581421
5-th fold: ch acc = 0.6499999761581421, en acc = 0.6833333373069763
6-th fold: ch acc = 0.6833333373069763, en acc = 0.699999988079071
7-th fold: ch acc = 0.5833333134651184, en acc = 0.4833333194255829
8-th fold: ch acc = 0.550000011920929, en acc = 0.6166666746139526
9-th fold: ch acc = 0.6166666746139526, en acc = 0.5333333611488342
average ch acc = 0.6083333492279053, average en acc = 0.6133333444595337


### 1 layer, dropout = 0.1

In [7]:
model_ch, model_en = train_and_test_k_fold_cross_val(hidden_num=1, dropout_p=0.1, lr=0.5, epoch_num=1000, debug_mode=False)

0-th fold: ch acc = 0.6666666865348816, en acc = 0.6166666746139526
1-th fold: ch acc = 0.5833333134651184, en acc = 0.5
2-th fold: ch acc = 0.550000011920929, en acc = 0.46666666865348816
3-th fold: ch acc = 0.6166666746139526, en acc = 0.5833333134651184
4-th fold: ch acc = 0.6000000238418579, en acc = 0.5333333611488342
5-th fold: ch acc = 0.5333333611488342, en acc = 0.5833333134651184
6-th fold: ch acc = 0.6000000238418579, en acc = 0.699999988079071
7-th fold: ch acc = 0.6000000238418579, en acc = 0.6166666746139526
8-th fold: ch acc = 0.699999988079071, en acc = 0.6000000238418579
9-th fold: ch acc = 0.6833333373069763, en acc = 0.6166666746139526
average ch acc = 0.6133332848548889, average en acc = 0.5816666483879089


### 3 layer, dropout = 0.1

In [8]:
model_ch, model_en = train_and_test_k_fold_cross_val(hidden_num=3, dropout_p=0.1, lr=0.5, epoch_num=1000, debug_mode=False)

0-th fold: ch acc = 0.6666666865348816, en acc = 0.6000000238418579
1-th fold: ch acc = 0.6833333373069763, en acc = 0.5666666626930237
2-th fold: ch acc = 0.5833333134651184, en acc = 0.6166666746139526
3-th fold: ch acc = 0.699999988079071, en acc = 0.6000000238418579
4-th fold: ch acc = 0.6499999761581421, en acc = 0.5833333134651184
5-th fold: ch acc = 0.5166666507720947, en acc = 0.6166666746139526
6-th fold: ch acc = 0.75, en acc = 0.6333333253860474
7-th fold: ch acc = 0.6499999761581421, en acc = 0.5833333134651184
8-th fold: ch acc = 0.46666666865348816, en acc = 0.5833333134651184
9-th fold: ch acc = 0.5666666626930237, en acc = 0.5333333611488342
average ch acc = 0.6233333349227905, average en acc = 0.5916666984558105


### 10 layer, dropout = 0.1

In [9]:
model_ch, model_en = train_and_test_k_fold_cross_val(hidden_num=10, dropout_p=0.1, lr=0.5, epoch_num=1000, debug_mode=False)

0-th fold: ch acc = 0.6666666865348816, en acc = 0.6333333253860474
1-th fold: ch acc = 0.6166666746139526, en acc = 0.5166666507720947
2-th fold: ch acc = 0.5833333134651184, en acc = 0.550000011920929
3-th fold: ch acc = 0.5666666626930237, en acc = 0.5166666507720947
4-th fold: ch acc = 0.6499999761581421, en acc = 0.5833333134651184
5-th fold: ch acc = 0.6333333253860474, en acc = 0.5666666626930237
6-th fold: ch acc = 0.6166666746139526, en acc = 0.6333333253860474
7-th fold: ch acc = 0.5833333134651184, en acc = 0.6666666865348816
8-th fold: ch acc = 0.550000011920929, en acc = 0.6666666865348816
9-th fold: ch acc = 0.7666666507720947, en acc = 0.699999988079071
average ch acc = 0.6233333349227905, average en acc = 0.6033332943916321


### check "very different"

### get {id : (ch_label, en_label)}

In [10]:
# using 1 layer, dropout = None

model_ch, model_en = train_and_test_k_fold_cross_val(hidden_num=1, dropout_p=None, lr=0.5, epoch_num=1000, debug_mode=False)

0-th fold: ch acc = 0.5333333611488342, en acc = 0.6166666746139526
1-th fold: ch acc = 0.6499999761581421, en acc = 0.5666666626930237
2-th fold: ch acc = 0.6333333253860474, en acc = 0.5333333611488342
3-th fold: ch acc = 0.6499999761581421, en acc = 0.6333333253860474
4-th fold: ch acc = 0.6499999761581421, en acc = 0.6166666746139526
5-th fold: ch acc = 0.6000000238418579, en acc = 0.6333333253860474
6-th fold: ch acc = 0.6833333373069763, en acc = 0.699999988079071
7-th fold: ch acc = 0.6666666865348816, en acc = 0.6499999761581421
8-th fold: ch acc = 0.6499999761581421, en acc = 0.5833333134651184
9-th fold: ch acc = 0.6833333373069763, en acc = 0.5666666626930237
average ch acc = 0.6399999856948853, average en acc = 0.6100000143051147


In [11]:
def get_pred(model, X_test):
    model.eval()
    test_size, input_dim = X_test.shape
    inputs = torch.tensor(X_test).float()
    pred_vals, pred_labels  = model(inputs)
    return pred_labels.detach().numpy().tolist()

In [12]:
arr = data_reader.read()
X = arr[:, :-2]
Y = arr[:, -2:]

pred_ch = get_pred(model_ch, X)
pred_en = get_pred(model_en, X)
very_diff_id_pred = set([i for i in range(len(pred_ch)) if pred_ch[i] != pred_en[i]])

### compare to "label_d_z"

In [13]:
import pandas as pd
df = pd.read_csv("../pol600withLabel.csv", encoding="utf-8", index_col=2)
label_d_z = df["label_d_z"]

In [14]:
very_diff_id_label = set([i for i in range(len(label_d_z)) if label_d_z.get(i) == 1])

In [15]:
p = len(very_diff_id_pred - very_diff_id_label) / len(very_diff_id_pred)
r = len(very_diff_id_label - very_diff_id_pred) / len(very_diff_id_label)
f = 2*p*r / (p+r)
print(p, r, f)

0.8440366972477065 0.6136363636363636 0.7106279502217137


In [16]:
len(very_diff_id_pred)

218

---

## data filtered on value 0.3

### 1 layer, no dropout

In [34]:
model_ch, model_en = train_and_test_k_fold_cross_val(filter_mode="value", filter_val=0.3, 
                                                     hidden_num=1, dropout_p=None, lr=0.5, epoch_num=1000, debug_mode=False)

training set size=211, test set size=24
0-th fold: ch acc = 0.7916666865348816, en acc = 0.5
training set size=211, test set size=24
1-th fold: ch acc = 0.6666666865348816, en acc = 0.6666666865348816
training set size=211, test set size=24
2-th fold: ch acc = 0.625, en acc = 0.5833333134651184
training set size=211, test set size=24
3-th fold: ch acc = 0.7916666865348816, en acc = 0.7083333134651184
training set size=211, test set size=24
4-th fold: ch acc = 0.625, en acc = 0.6666666865348816
training set size=212, test set size=23
5-th fold: ch acc = 0.6521739363670349, en acc = 0.6086956262588501
training set size=212, test set size=23
6-th fold: ch acc = 0.6521739363670349, en acc = 0.6086956262588501
training set size=212, test set size=23
7-th fold: ch acc = 0.5652173757553101, en acc = 0.6521739363670349
training set size=212, test set size=23
8-th fold: ch acc = 0.782608687877655, en acc = 0.8260869383811951
training set size=212, test set size=23
9-th fold: ch acc = 0.47826087

### 3 layer, no dropout

In [35]:
model_ch, model_en = train_and_test_k_fold_cross_val(filter_mode="value", filter_val=0.3, 
                                                     hidden_num=3, dropout_p=None, lr=0.5, epoch_num=1000, debug_mode=False)

training set size=211, test set size=24
0-th fold: ch acc = 0.5, en acc = 0.5416666865348816
training set size=211, test set size=24
1-th fold: ch acc = 0.7083333134651184, en acc = 0.625
training set size=211, test set size=24
2-th fold: ch acc = 0.7916666865348816, en acc = 0.75
training set size=211, test set size=24
3-th fold: ch acc = 0.8333333134651184, en acc = 0.7916666865348816
training set size=211, test set size=24
4-th fold: ch acc = 0.5416666865348816, en acc = 0.5833333134651184
training set size=212, test set size=23
5-th fold: ch acc = 0.52173912525177, en acc = 0.695652186870575
training set size=212, test set size=23
6-th fold: ch acc = 0.6521739363670349, en acc = 0.47826087474823
training set size=212, test set size=23
7-th fold: ch acc = 0.8260869383811951, en acc = 0.6521739363670349
training set size=212, test set size=23
8-th fold: ch acc = 0.6521739363670349, en acc = 0.6521739363670349
training set size=212, test set size=23
9-th fold: ch acc = 0.5217391252517

### 1 layer, dropout = 0.1

In [36]:
model_ch, model_en = train_and_test_k_fold_cross_val(filter_mode="value", filter_val=0.3, 
                                                     hidden_num=1, dropout_p=0.1, lr=0.5, epoch_num=1000, debug_mode=False)

training set size=211, test set size=24
0-th fold: ch acc = 0.6666666865348816, en acc = 0.5416666865348816
training set size=211, test set size=24
1-th fold: ch acc = 0.625, en acc = 0.6666666865348816
training set size=211, test set size=24
2-th fold: ch acc = 0.6666666865348816, en acc = 0.625
training set size=211, test set size=24
3-th fold: ch acc = 0.75, en acc = 0.625
training set size=211, test set size=24
4-th fold: ch acc = 0.6666666865348816, en acc = 0.75
training set size=212, test set size=23
5-th fold: ch acc = 0.695652186870575, en acc = 0.6521739363670349
training set size=212, test set size=23
6-th fold: ch acc = 0.739130437374115, en acc = 0.6086956262588501
training set size=212, test set size=23
7-th fold: ch acc = 0.739130437374115, en acc = 0.6086956262588501
training set size=212, test set size=23
8-th fold: ch acc = 0.6521739363670349, en acc = 0.695652186870575
training set size=212, test set size=23
9-th fold: ch acc = 0.52173912525177, en acc = 0.6956521868

### 3 layer, dropout = 0.1

In [37]:
model_ch, model_en = train_and_test_k_fold_cross_val(filter_mode="value", filter_val=0.3, 
                                                     hidden_num=3, dropout_p=0.1, lr=0.5, epoch_num=1000, debug_mode=False)

training set size=211, test set size=24
0-th fold: ch acc = 0.5833333134651184, en acc = 0.625
training set size=211, test set size=24
1-th fold: ch acc = 0.8333333134651184, en acc = 0.5833333134651184
training set size=211, test set size=24
2-th fold: ch acc = 0.6666666865348816, en acc = 0.5833333134651184
training set size=211, test set size=24
3-th fold: ch acc = 0.875, en acc = 0.75
training set size=211, test set size=24
4-th fold: ch acc = 0.6666666865348816, en acc = 0.6666666865348816
training set size=212, test set size=23
5-th fold: ch acc = 0.6086956262588501, en acc = 0.5652173757553101
training set size=212, test set size=23
6-th fold: ch acc = 0.739130437374115, en acc = 0.6086956262588501
training set size=212, test set size=23
7-th fold: ch acc = 0.739130437374115, en acc = 0.739130437374115
training set size=212, test set size=23
8-th fold: ch acc = 0.6521739363670349, en acc = 0.6086956262588501
training set size=212, test set size=23
9-th fold: ch acc = 0.608695626

### 10 layer, dropout = 0.1

In [38]:
model_ch, model_en = train_and_test_k_fold_cross_val(filter_mode="value", filter_val=0.3, 
                                                     hidden_num=10, dropout_p=0.1, lr=0.5, epoch_num=1000, debug_mode=False)

training set size=211, test set size=24
0-th fold: ch acc = 0.7916666865348816, en acc = 0.5416666865348816
training set size=211, test set size=24
1-th fold: ch acc = 0.75, en acc = 0.7916666865348816
training set size=211, test set size=24
2-th fold: ch acc = 0.7083333134651184, en acc = 0.5
training set size=211, test set size=24
3-th fold: ch acc = 0.5833333134651184, en acc = 0.75
training set size=211, test set size=24
4-th fold: ch acc = 0.5833333134651184, en acc = 0.625
training set size=212, test set size=23
5-th fold: ch acc = 0.47826087474823, en acc = 0.695652186870575
training set size=212, test set size=23
6-th fold: ch acc = 0.5652173757553101, en acc = 0.43478259444236755
training set size=212, test set size=23
7-th fold: ch acc = 0.739130437374115, en acc = 0.782608687877655
training set size=212, test set size=23
8-th fold: ch acc = 0.6521739363670349, en acc = 0.6521739363670349
training set size=212, test set size=23
9-th fold: ch acc = 0.6086956262588501, en acc =

### check "very different"

### get {id : (ch_label, en_label)}

In [39]:
# using 1 layer, dropout = None

model_ch, model_en = train_and_test_k_fold_cross_val(filter_mode="value", filter_val=0.3, 
                                                     hidden_num=1, dropout_p=None, lr=0.5, epoch_num=1000, debug_mode=False)

training set size=211, test set size=24
0-th fold: ch acc = 0.7083333134651184, en acc = 0.5833333134651184
training set size=211, test set size=24
1-th fold: ch acc = 0.7083333134651184, en acc = 0.625
training set size=211, test set size=24
2-th fold: ch acc = 0.5833333134651184, en acc = 0.7083333134651184
training set size=211, test set size=24
3-th fold: ch acc = 0.6666666865348816, en acc = 0.7083333134651184
training set size=211, test set size=24
4-th fold: ch acc = 0.6666666865348816, en acc = 0.6666666865348816
training set size=212, test set size=23
5-th fold: ch acc = 0.5652173757553101, en acc = 0.695652186870575
training set size=212, test set size=23
6-th fold: ch acc = 0.6086956262588501, en acc = 0.6086956262588501
training set size=212, test set size=23
7-th fold: ch acc = 0.782608687877655, en acc = 0.6086956262588501
training set size=212, test set size=23
8-th fold: ch acc = 0.695652186870575, en acc = 0.695652186870575
training set size=212, test set size=23
9-th 

In [40]:
def get_pred(model, X_test):
    model.eval()
    test_size, input_dim = X_test.shape
    inputs = torch.tensor(X_test).float()
    pred_vals, pred_labels  = model(inputs)
    return pred_labels.detach().numpy().tolist()

In [41]:
arr = data_reader.read()
X = arr[:, :-2]
Y = arr[:, -2:]

pred_ch = get_pred(model_ch, X)
pred_en = get_pred(model_en, X)
very_diff_id_pred = set([i for i in range(len(pred_ch)) if pred_ch[i] != pred_en[i]])

### compare to "label_d_z"

In [42]:
import pandas as pd
df = pd.read_csv("../pol600withLabel.csv", encoding="utf-8", index_col=2)
label_d_z = df["label_d_z"]

In [43]:
very_diff_id_label = set([i for i in range(len(label_d_z)) if label_d_z.get(i) == 1])

In [44]:
p = len(very_diff_id_pred - very_diff_id_label) / len(very_diff_id_pred)
r = len(very_diff_id_label - very_diff_id_pred) / len(very_diff_id_label)
f = 2*p*r / (p+r)
print(p, r, f)

0.8263157894736842 0.625 0.7116953762466002


In [45]:
len(very_diff_id_pred)

190

---

## data filtered on percentile 30

### 1 layer, no dropout

In [5]:
model_ch, model_en = train_and_test_k_fold_cross_val(filter_mode="percentile", filter_val=30, 
                                                     hidden_num=1, dropout_p=None, lr=0.5, epoch_num=1000, debug_mode=False)

filtering data by percentile
training set size=213, test set size=24
0-th fold: ch acc = 0.75, en acc = 0.75
training set size=213, test set size=24
1-th fold: ch acc = 0.6666666865348816, en acc = 0.5
training set size=213, test set size=24
2-th fold: ch acc = 0.5416666865348816, en acc = 0.625
training set size=213, test set size=24
3-th fold: ch acc = 0.7916666865348816, en acc = 0.7083333134651184
training set size=213, test set size=24
4-th fold: ch acc = 0.6666666865348816, en acc = 0.7083333134651184
training set size=213, test set size=24
5-th fold: ch acc = 0.8333333134651184, en acc = 0.6666666865348816
training set size=213, test set size=24
6-th fold: ch acc = 0.75, en acc = 0.5
training set size=214, test set size=23
7-th fold: ch acc = 0.739130437374115, en acc = 0.6521739363670349
training set size=214, test set size=23
8-th fold: ch acc = 0.5652173757553101, en acc = 0.695652186870575
training set size=214, test set size=23
9-th fold: ch acc = 0.5652173757553101, en acc

### 3 layer, no dropout

In [6]:
model_ch, model_en = train_and_test_k_fold_cross_val(filter_mode="percentile", filter_val=30, 
                                                     hidden_num=3, dropout_p=None, lr=0.5, epoch_num=1000, debug_mode=False)

filtering data by percentile
training set size=213, test set size=24
0-th fold: ch acc = 0.7083333134651184, en acc = 0.625
training set size=213, test set size=24
1-th fold: ch acc = 0.5, en acc = 0.5833333134651184
training set size=213, test set size=24
2-th fold: ch acc = 0.75, en acc = 0.7083333134651184
training set size=213, test set size=24
3-th fold: ch acc = 0.8333333134651184, en acc = 0.8333333134651184
training set size=213, test set size=24
4-th fold: ch acc = 0.625, en acc = 0.6666666865348816
training set size=213, test set size=24
5-th fold: ch acc = 0.625, en acc = 0.625
training set size=213, test set size=24
6-th fold: ch acc = 0.7916666865348816, en acc = 0.6666666865348816
training set size=214, test set size=23
7-th fold: ch acc = 0.782608687877655, en acc = 0.695652186870575
training set size=214, test set size=23
8-th fold: ch acc = 0.695652186870575, en acc = 0.739130437374115
training set size=214, test set size=23
9-th fold: ch acc = 0.695652186870575, en ac

### 1 layer, dropout = 0.1

In [7]:
model_ch, model_en = train_and_test_k_fold_cross_val(filter_mode="percentile", filter_val=30, 
                                                     hidden_num=1, dropout_p=0.1, lr=0.5, epoch_num=1000, debug_mode=False)

filtering data by percentile
training set size=213, test set size=24
0-th fold: ch acc = 0.625, en acc = 0.625
training set size=213, test set size=24
1-th fold: ch acc = 0.5416666865348816, en acc = 0.4583333432674408
training set size=213, test set size=24
2-th fold: ch acc = 0.7083333134651184, en acc = 0.5833333134651184
training set size=213, test set size=24
3-th fold: ch acc = 0.7083333134651184, en acc = 0.5416666865348816
training set size=213, test set size=24
4-th fold: ch acc = 0.625, en acc = 0.75
training set size=213, test set size=24
5-th fold: ch acc = 0.6666666865348816, en acc = 0.7083333134651184
training set size=213, test set size=24
6-th fold: ch acc = 0.75, en acc = 0.6666666865348816
training set size=214, test set size=23
7-th fold: ch acc = 0.739130437374115, en acc = 0.6521739363670349
training set size=214, test set size=23
8-th fold: ch acc = 0.6086956262588501, en acc = 0.782608687877655
training set size=214, test set size=23
9-th fold: ch acc = 0.739130

### 3 layer, dropout = 0.1

In [8]:
model_ch, model_en = train_and_test_k_fold_cross_val(filter_mode="percentile", filter_val=30, 
                                                     hidden_num=3, dropout_p=0.1, lr=0.5, epoch_num=1000, debug_mode=False)

filtering data by percentile
training set size=213, test set size=24
0-th fold: ch acc = 0.625, en acc = 0.5833333134651184
training set size=213, test set size=24
1-th fold: ch acc = 0.5416666865348816, en acc = 0.5
training set size=213, test set size=24
2-th fold: ch acc = 0.75, en acc = 0.4583333432674408
training set size=213, test set size=24
3-th fold: ch acc = 0.75, en acc = 0.6666666865348816
training set size=213, test set size=24
4-th fold: ch acc = 0.7916666865348816, en acc = 0.7083333134651184
training set size=213, test set size=24
5-th fold: ch acc = 0.625, en acc = 0.7083333134651184
training set size=213, test set size=24
6-th fold: ch acc = 0.6666666865348816, en acc = 0.6666666865348816
training set size=214, test set size=23
7-th fold: ch acc = 0.5652173757553101, en acc = 0.6521739363670349
training set size=214, test set size=23
8-th fold: ch acc = 0.6086956262588501, en acc = 0.782608687877655
training set size=214, test set size=23
9-th fold: ch acc = 0.6956521

### 10 layer, dropout = 0.1

In [9]:
model_ch, model_en = train_and_test_k_fold_cross_val(filter_mode="percentile", filter_val=30, 
                                                     hidden_num=10, dropout_p=0.1, lr=0.5, epoch_num=1000, debug_mode=False)

filtering data by percentile
training set size=213, test set size=24
0-th fold: ch acc = 0.4583333432674408, en acc = 0.75
training set size=213, test set size=24
1-th fold: ch acc = 0.5833333134651184, en acc = 0.4583333432674408
training set size=213, test set size=24
2-th fold: ch acc = 0.5416666865348816, en acc = 0.2916666567325592
training set size=213, test set size=24
3-th fold: ch acc = 0.625, en acc = 0.6666666865348816
training set size=213, test set size=24
4-th fold: ch acc = 0.75, en acc = 0.6666666865348816
training set size=213, test set size=24
5-th fold: ch acc = 0.6666666865348816, en acc = 0.75
training set size=213, test set size=24
6-th fold: ch acc = 0.75, en acc = 0.7083333134651184
training set size=214, test set size=23
7-th fold: ch acc = 0.782608687877655, en acc = 0.695652186870575
training set size=214, test set size=23
8-th fold: ch acc = 0.695652186870575, en acc = 0.6086956262588501
training set size=214, test set size=23
9-th fold: ch acc = 0.652173936

### check "very different"

### get {id : (ch_label, en_label)}

In [10]:
# using 1 layer, dropout = None

model_ch, model_en = train_and_test_k_fold_cross_val(filter_mode="percentile", filter_val=30, 
                                                     hidden_num=1, dropout_p=None, lr=0.5, epoch_num=1000, debug_mode=False)

filtering data by percentile
training set size=213, test set size=24
0-th fold: ch acc = 0.5, en acc = 0.5833333134651184
training set size=213, test set size=24
1-th fold: ch acc = 0.4583333432674408, en acc = 0.5833333134651184
training set size=213, test set size=24
2-th fold: ch acc = 0.5833333134651184, en acc = 0.6666666865348816
training set size=213, test set size=24
3-th fold: ch acc = 0.5833333134651184, en acc = 0.625
training set size=213, test set size=24
4-th fold: ch acc = 0.6666666865348816, en acc = 0.75
training set size=213, test set size=24
5-th fold: ch acc = 0.7083333134651184, en acc = 0.7083333134651184
training set size=213, test set size=24
6-th fold: ch acc = 0.7916666865348816, en acc = 0.5416666865348816
training set size=214, test set size=23
7-th fold: ch acc = 0.6521739363670349, en acc = 0.6086956262588501
training set size=214, test set size=23
8-th fold: ch acc = 0.695652186870575, en acc = 0.6521739363670349
training set size=214, test set size=23
9-

In [11]:
def get_pred(model, X_test):
    model.eval()
    test_size, input_dim = X_test.shape
    inputs = torch.tensor(X_test).float()
    pred_vals, pred_labels  = model(inputs)
    return pred_labels.detach().numpy().tolist()

In [12]:
arr = data_reader.read()
X = arr[:, :-2]
Y = arr[:, -2:]

pred_ch = get_pred(model_ch, X)
pred_en = get_pred(model_en, X)
very_diff_id_pred = set([i for i in range(len(pred_ch)) if pred_ch[i] != pred_en[i]])

### compare to "label_d_z"

In [13]:
import pandas as pd
df = pd.read_csv("../pol600withLabel.csv", encoding="utf-8", index_col=2)
label_d_z = df["label_d_z"]

In [14]:
very_diff_id_label = set([i for i in range(len(label_d_z)) if label_d_z.get(i) == 1])

In [15]:
p = len(very_diff_id_pred - very_diff_id_label) / len(very_diff_id_pred)
r = len(very_diff_id_label - very_diff_id_pred) / len(very_diff_id_label)
f = 2*p*r / (p+r)
print(p, r, f)

0.825 0.6022727272727273 0.6962579617834396


In [16]:
len(very_diff_id_pred)

200